# Using MobileNet to classify traffic signs

We will use [MobileNet](https://arxiv.org/abs/1704.04861) and fine-tune it to correctly classify a dataset of traffic signs.

MobileNet is a small CNN originaly developed for mobile phones and other small devices to be fast and lightweight.

--------------------

Load necessary packages and libraries

In [1]:
import keras
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.utils import get_file, load_img, img_to_array
import numpy as np
from IPython.display import Image

Load MobileNet

In [2]:
mobile = keras.applications.mobilenet.MobileNet()

In [3]:
def prepare_image(file):
    img = load_img(file, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

## Testing MobileNet on dog images

Let's try some tests on images of different dog breeds

In [ ]:
Image(data='https://upload.wikimedia.org/wikipedia/commons/4/4f/German-shepherd-4040871920._%282%29.jpg') 

In [ ]:
preprocessed_image = prepare_image(get_file('German-shepperd.jpg',origin='https://upload.wikimedia.org/wikipedia/commons/4/4f/German-shepherd-4040871920._%282%29.jpg'))
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

In [ ]:
Image(data='https://upload.wikimedia.org/wikipedia/commons/d/d4/Labrador_Retriever_-_Yellow.JPG')

In [ ]:
preprocessed_image = prepare_image(get_file('Labrador.jpg',origin='https://upload.wikimedia.org/wikipedia/commons/d/d4/Labrador_Retriever_-_Yellow.JPG'))
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

It works pretty well, you can try here some different pictures if you're curious.

## TODO - test on Traffic signs

Now let's test the network on some images of traffic signs. We will work with `Stop`, `Speedlimit` and `Crosswalk`.
Please use the code above as a template and try to find some images of the traffic signs and test the network on it.

In [1]:
# TODO

## Create traffic sign dataset

Lets now manipulate MobileNet top few layers and employ transfer learning. To do this, we need to train it on some images. We will train it on `Stop`, `Speedlimit` and `Crosswalk` traffic signs. But instead of manually downloading images of them, let's use Google Image Search and pull the images.

In [8]:
! pip install simple_image_download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from simple_image_download import simple_image_download as simp

In [10]:
response = simp.simple_image_download

In [ ]:
response().download('stop traffic sign', 150)

In [ ]:
response().download('maximum speed traffic sign', 150)

In [13]:
response().download('crosswalk traffic sign', 150)

Check size of the images and remove the ones with low quality

In [14]:
import os
from PIL import Image

def check_pictures(directory):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        try:
            img = Image.open(f)
            wid, hgt = img.size
            if wid < 300 or hgt < 300:
                os.remove(f)
        except:
            os.remove(f)
        

            
check_pictures("simple_images/stop traffic sign")
check_pictures("simple_images/maximum speed traffic sign")
check_pictures("simple_images/crosswalk traffic sign")

## Prepare model

Let's now use MobileNet, freeze the base layers and let's add and train the top few layers.

In [ ]:
base_model = MobileNet(weights='imagenet',include_top=False) # imports the mobilenet model and discards the last layer.

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) # we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024,activation='relu')(x) # dense layer 2
x = Dense(512,activation='relu')(x) # dense layer 3
preds = Dense(3,activation='softmax')(x) # final layer with softmax activation (this gives us probability), 3 outputs for 3 labels

In [22]:
model = Model(inputs=base_model.input,outputs=preds)

In [ ]:
model.summary()

We will use pre-trained weights as the model has been trained already on the ImageNet dataset. We ensure all the weights are non-trainable, we will only train the last few layers.

In [24]:
for layer in model.layers[:80]:
    layer.trainable = False
for layer in model.layers[80:]:
    layer.trainable = True

Now lets load the training data into the ImageDataGenerator. Specify path, and it automatically sends the data for training in batches, simplifying the code.

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

batch_size = 32

train_generator = datagen.flow_from_directory('simple_images',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='training')
validation_generator = datagen.flow_from_directory('simple_images',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='validation')

Now let's do the real training

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(train_generator,
                    steps_per_epoch=train_generator.n // batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // batch_size,
                    epochs=5)


## TODO test the model

Model is now trained. Now let's test some other input images to check the predictions.

In [28]:
# TODO

In [ ]:
# see which label corresponds to which class
validation_generator.class_indices